# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('census_data.csv')

In [4]:
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [5]:
data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [8]:
data['income_bracket'] = data['income_bracket'].apply(label_fix)

In [11]:
data = data.dropna(axis = 0)

In [12]:
cols_to_norm = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']


In [13]:
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

### Perform a Train Test Split on the Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = data.drop(['income_bracket'], axis = 1)
Y = data['income_bracket']

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [17]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [18]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [36]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size = data['workclass'].unique().size)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size = data['education'].unique().size)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size = data['marital_status'].unique().size)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size = data['occupation'].unique().size)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size = data['relationship'].unique().size)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size = data['race'].unique().size)
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender', ['Female', 'Male'])
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size = data['native_country'].unique().size)


** Create the continuous feature_columns for the continuous values using numeric_column **

In [37]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_week = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [38]:
feat_cols = [age, workclass, education, education_num, marital_status, occupation, relationship, race, gender, capital_gain, capital_loss, hours_week, native_country]

In [39]:
feat_cols

[_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _HashedCategoricalColumn(key='workclass', hash_bucket_size=9, dtype=tf.string),
 _HashedCategoricalColumn(key='education', hash_bucket_size=16, dtype=tf.string),
 _NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _HashedCategoricalColumn(key='marital_status', hash_bucket_size=7, dtype=tf.string),
 _HashedCategoricalColumn(key='occupation', hash_bucket_size=15, dtype=tf.string),
 _HashedCategoricalColumn(key='relationship', hash_bucket_size=6, dtype=tf.string),
 _HashedCategoricalColumn(key='race', hash_bucket_size=5, dtype=tf.string),
 _VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_loss', shape=(1,), default_

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [40]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=Y_train, batch_size=10, num_epochs=1000, shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [41]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\dns\\AppData\\Local\\Temp\\tmpyz9xfvxr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E8D37C6BE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [43]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\dns\AppData\Local\Temp\tmpyz9xfvxr\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\dns\AppData\Local\Temp\tmpyz9xfvxr\model.ckpt.
INFO:tensorflow:loss = 1.14867, step = 5001
INFO:tensorflow:global_step/sec: 104.231
INFO:tensorflow:loss = 2.80175, step = 5101 (0.969 sec)
INFO:tensorflow:global_step/sec: 220.4
INFO:tensorflow:loss = 1.9077, step = 5201 (0.449 sec)
INFO:tensorflow:global_step/sec: 237.114
INFO:tensorflow:loss = 2.11552, step = 5301 (0.425 sec)
INFO:tensorflow:global_step/sec: 203.791
INFO:tensorflow:loss = 2.46476, step = 5401 (0.497 sec)
INFO:tensorflow:global_step/sec: 192.797
INFO:tensorflow:loss = 2.44302, step = 5501 (0.509 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 5.66582, step = 13001 (0.426 sec)
INFO:tensorflow:global_step/sec: 234.756
INFO:tensorflow:loss = 3.47659, step = 13101 (0.434 sec)
INFO:tensorflow:global_step/sec: 212.445
INFO:tensorflow:loss = 3.2944, step = 13201 (0.468 sec)
INFO:tensorflow:global_step/sec: 231.133
INFO:tensorflow:loss = 2.56148, step = 13301 (0.437 sec)
INFO:tensorflow:global_step/sec: 214.102
INFO:tensorflow:loss = 2.37604, step = 13401 (0.466 sec)
INFO:tensorflow:global_step/sec: 229.498
INFO:tensorflow:loss = 3.41493, step = 13501 (0.433 sec)
INFO:tensorflow:global_step/sec: 225.873
INFO:tensorflow:loss = 3.69182, step = 13601 (0.446 sec)
INFO:tensorflow:global_step/sec: 238.242
INFO:tensorflow:loss = 2.70233, step = 13701 (0.424 sec)
INFO:tensorflow:global_step/sec: 211.1
INFO:tensorflow:loss = 2.89849, step = 13801 (0.469 sec)
INFO:tensorflow:global_step/sec: 226.897
INFO:tensorflow:loss = 2.22214, step = 13901 (0.442 sec)
INFO:tensorflow:global_step/sec: 218.475
INFO:tensorflow:loss = 

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [45]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=Y_test, batch_size=10, num_epochs=1, shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [47]:
predictions = model.predict(eval_input_func)

In [49]:
predicts = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\dns\AppData\Local\Temp\tmpyz9xfvxr\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [50]:
for pred in predicts:
    print(pred)

{'logits': array([-1.01636624], dtype=float32), 'logistic': array([ 0.26573581], dtype=float32), 'probabilities': array([ 0.73426419,  0.26573581], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.66870999], dtype=float32), 'logistic': array([ 0.02487482], dtype=float32), 'probabilities': array([ 0.97512519,  0.02487482], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.03836131], dtype=float32), 'logistic': array([ 0.26146632], dtype=float32), 'probabilities': array([ 0.73853368,  0.26146629], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.19620991], dtype=float32), 'logistic': array([ 0.0393086], dtype=float32), 'probabilities': array([ 0.96069133,  0.0393086 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.63718176], d

{'logits': array([-3.11627102], dtype=float32), 'logistic': array([ 0.04244106], dtype=float32), 'probabilities': array([ 0.95755893,  0.04244106], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.701401], dtype=float32), 'logistic': array([ 0.02409406], dtype=float32), 'probabilities': array([ 0.9759059 ,  0.02409405], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.39711207], dtype=float32), 'logistic': array([ 0.40200639], dtype=float32), 'probabilities': array([ 0.59799361,  0.40200639], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.2416842], dtype=float32), 'logistic': array([ 0.09606919], dtype=float32), 'probabilities': array([ 0.90393078,  0.09606919], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.39257932], dty

{'logits': array([-1.48068976], dtype=float32), 'logistic': array([ 0.18532325], dtype=float32), 'probabilities': array([ 0.8146767 ,  0.18532325], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.41719526], dtype=float32), 'logistic': array([ 0.3971881], dtype=float32), 'probabilities': array([ 0.60281187,  0.3971881 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.89701349], dtype=float32), 'logistic': array([ 0.28966463], dtype=float32), 'probabilities': array([ 0.71033543,  0.28966466], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.39755636], dtype=float32), 'logistic': array([ 0.40189961], dtype=float32), 'probabilities': array([ 0.59810036,  0.40189955], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.28325057], d

{'logits': array([-1.81867838], dtype=float32), 'logistic': array([ 0.13959254], dtype=float32), 'probabilities': array([ 0.86040747,  0.13959253], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.95726287], dtype=float32), 'logistic': array([ 0.27742654], dtype=float32), 'probabilities': array([ 0.72257346,  0.27742657], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 0.0351643], dtype=float32), 'logistic': array([ 0.50879019], dtype=float32), 'probabilities': array([ 0.49120986,  0.50879019], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-0.24948317], dtype=float32), 'logistic': array([ 0.43795073], dtype=float32), 'probabilities': array([ 0.56204927,  0.4379507 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 2.52984977], d

{'logits': array([-2.69649363], dtype=float32), 'logistic': array([ 0.06318057], dtype=float32), 'probabilities': array([ 0.93681943,  0.06318058], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 0.47933501], dtype=float32), 'logistic': array([ 0.61759084], dtype=float32), 'probabilities': array([ 0.38240916,  0.61759084], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.32693267], dtype=float32), 'logistic': array([ 0.20966719], dtype=float32), 'probabilities': array([ 0.79033279,  0.20966719], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.76340055], dtype=float32), 'logistic': array([ 0.00846428], dtype=float32), 'probabilities': array([ 0.99153566,  0.00846427], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.80711961], 

...

{'logits': array([-3.98969913], dtype=float32), 'logistic': array([ 0.01816906], dtype=float32), 'probabilities': array([ 0.98183089,  0.01816906], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.8105886], dtype=float32), 'logistic': array([ 0.05675466], dtype=float32), 'probabilities': array([ 0.94324535,  0.05675466], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.1096282], dtype=float32), 'logistic': array([ 0.24794018], dtype=float32), 'probabilities': array([ 0.75205982,  0.24794023], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.95405644], dtype=float32), 'logistic': array([ 0.27806976], dtype=float32), 'probabilities': array([ 0.72193021,  0.27806976], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.67041981], dt

{'logits': array([-1.64037585], dtype=float32), 'logistic': array([ 0.16241392], dtype=float32), 'probabilities': array([ 0.8375861 ,  0.16241392], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.51730251], dtype=float32), 'logistic': array([ 0.02882391], dtype=float32), 'probabilities': array([ 0.97117609,  0.02882391], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 2.15474796], dtype=float32), 'logistic': array([ 0.89611155], dtype=float32), 'probabilities': array([ 0.10388837,  0.89611155], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.07438707], dtype=float32), 'logistic': array([ 0.04417621], dtype=float32), 'probabilities': array([ 0.95582384,  0.04417622], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.37737226], 

{'logits': array([-2.00053811], dtype=float32), 'logistic': array([ 0.11914643], dtype=float32), 'probabilities': array([ 0.88085353,  0.11914643], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.22062445], dtype=float32), 'logistic': array([ 0.2278266], dtype=float32), 'probabilities': array([ 0.7721734 ,  0.22782658], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.77604866], dtype=float32), 'logistic': array([ 0.05863227], dtype=float32), 'probabilities': array([ 0.94136775,  0.05863227], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.6847102], dtype=float32), 'logistic': array([ 0.33521086], dtype=float32), 'probabilities': array([ 0.66478914,  0.33521083], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.04570484], dt

{'logits': array([-3.22234082], dtype=float32), 'logistic': array([ 0.0383336], dtype=float32), 'probabilities': array([ 0.96166635,  0.0383336 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.48222876], dtype=float32), 'logistic': array([ 0.00414281], dtype=float32), 'probabilities': array([ 0.99585718,  0.00414281], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.59704518], dtype=float32), 'logistic': array([ 0.06932883], dtype=float32), 'probabilities': array([ 0.93067116,  0.06932882], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.76135248], dtype=float32), 'logistic': array([ 0.3183527], dtype=float32), 'probabilities': array([ 0.6816473,  0.3183527], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 0.1370998], dtype

{'logits': array([-1.13842535], dtype=float32), 'logistic': array([ 0.24260959], dtype=float32), 'probabilities': array([ 0.75739044,  0.24260959], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.48748302], dtype=float32), 'logistic': array([ 0.07674034], dtype=float32), 'probabilities': array([ 0.92325968,  0.07674034], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 0.09275323], dtype=float32), 'logistic': array([ 0.52317166], dtype=float32), 'probabilities': array([ 0.47682828,  0.52317166], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-4.321877], dtype=float32), 'logistic': array([ 0.01310103], dtype=float32), 'probabilities': array([ 0.98689902,  0.01310103], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.35123658], dt

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [54]:
class_ids = []
for pred in predicts:
    class_ids.append(pred['class_ids'][0])

In [55]:
class_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [57]:
from sklearn import metrics

In [58]:
Y = class_ids
class_names = ['0', '1']
print(metrics.classification_report(Y_test, Y, target_names=class_names))

             precision    recall  f1-score   support

          0       0.85      0.94      0.89      7436
          1       0.72      0.47      0.57      2333

avg / total       0.82      0.83      0.82      9769



# Great Job!